<a href="https://colab.research.google.com/github/apschlissel/capstone/blob/main/Book_Description_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Analysis of the Description Field of the Data**

Read in the Data

In [ ]:
##Import Libraries
import string
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.tokenize import word_tokenize

import itertools
import scipy.sparse as sp
import numpy as np
import math

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
## Downloads
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
##Set Tools
stemmer = SnowballStemmer("english") 
wn = nltk.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("none")
stopwords.append("nan")

In [ ]:
## Read in Data
def file_input(filename):
    try:
        #making my dataframe from excel file and printing out
        data = pd.read_excel(filename, dtype='object')
        return data
        
    except Exception:
        #making my dataframe from csv file and printing out 
        data = pd.read_csv(filename, dtype='object')
        return data

In [ ]:
def co_occurance_matricies(data):
    
    columns = data.loc[:,data.columns != "Unique_id"]
    col_count_vectorizers = {}
    matricies = {}
    for col in columns:
        
        print(col)
        corpus = itertools.chain.from_iterable(data[col])
        vocab = set(corpus)
        varnames = tuple(sorted(vocab))
        document = data[col]
        rows, cols, vals = [], [], []
        voc2id = dict(zip(vocab, range(len(vocab))))
        id2voc = dict(zip(range(len(vocab)), vocab))
        for r, d in enumerate(document):
            for e in d:
                rows.append(r)
                cols.append(voc2id[e])
                vals.append(1)
        X = sp.csr_matrix((vals, (rows, cols)))
        Xc = (X.T * X)
        Xc.setdiag(0)
        Xc = Xc.todense()
        sums = np.sum(Xc, axis=0)
        sums = sums.tolist()
        sums = sums[0]
        total_count = sum(sums)
        Xc = sp.csr_matrix(Xc)
        min_num = min(sums)
        min_prob = min_num/total_count
        non_zeros = Xc.nonzero()
        row_coords = []
        col_coords = []
        data_coord = []
        for i in range(0, len(non_zeros[0])):
            print(i/len(non_zeros[0]))
            x = non_zeros[0][i]
            y = non_zeros[1][i]
            word = Xc[x, y]
            row_sum = sums[x]/total_count
            column_sum = sums[y]/total_count
            uniq_word_count = word/total_count
            #Pa = (column_sum**.75)/((column_sum**.75)+(min_prob**.75))
            wppmi = max(math.log2((uniq_word_count/(row_sum*column_sum))), 0)
            row_coords.append(x)
            col_coords.append(y)
            data_coord.append(wppmi)
            
        
        row_coords = np.array(row_coords) 
        col_coords = np.array(col_coords)
        data_coord = np.array(data_coord)
        max_wppmi = max(data_coord) + 5
        wppmi_matrix = sp.csr_matrix((data_coord, (row_coords, col_coords))
                                  ,shape = (max(non_zeros[0]+1), max(non_zeros[1]+1)))
        
        wppmi_matrix = wppmi_matrix.toarray()
        similarity_enc = {}
        i = 0
        for voc_word in voc2id:
            print(i/len(voc2id))
            i += 1
            mat_row = wppmi_matrix[voc2id[voc_word]]
            sorted_mat = np.sort(mat_row)
            sorted_mat = sorted_mat[::-1]
            top5 = sorted_mat[:4]
            top5_index = {}
            mat_row = mat_row.tolist()
            
            for num in top5:
                index = mat_row.index(num)
                mat_row.pop(index)
                index_word = id2voc[index]
                top5_index[index_word] = (index, num)
                
            similarity_enc[voc_word] = top5_index
            
    return similarity_enc

In [ ]:
def count_unique_mentions(dat):
    
    

    total_count = {}
    
    for col in dat:
        col_count = {}
        col_counts = {}
        docs = dat[col]
        count = 0
        for doc in docs:
            sentence = []
            for char in doc:
                if char in sentence:
                    continue
                else:
                    if char in total_count:
                        total_count[char] = total_count[char] + 1
                    else:
                        total_count[char] = 1
                    if char in col_count:
                        col_count[char] = col_count[char] + 1
                    else:
                        col_count[char] = 1
        
        col_counts[col] = col_count
                    

        
    return [total_count, col_count]

In [ ]:
orig_df = file_input("./books_filtered_colsreduced.csv")
df = orig_df[['description']].copy()
data = df
data = data.astype(str)
df = df.astype(str)
index = 0
vocab = []
for col in df:
    index += 1
    df[col] = df[col].str.lower()
    df[col] = df[col].apply(word_tokenize)
    data[col] = data[col].apply(word_tokenize)
    df[col] = df[col].apply(lambda words: [word for word in words if word not in stopwords])
    df[col] = df[col].apply(lambda x: [stemmer.stem(y) for y in x])
    corpus = list(itertools.chain.from_iterable(df[col]))
    vocab = vocab + corpus
    print(str(index) + 'out of' + str(df.shape[1]) + 'columns processed')
      
vocab = set(vocab)
index = 0

1out of1columns processed


In [ ]:
word_similarities = co_occurance_matricies(df)

description


In [ ]:
tot_count, col_count = count_unique_mentions(data)

NameError: ignored